In [1]:
import numpy as np
import pickle

%matplotlib inline

In [2]:
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# constants and paths

In [3]:
Emin = 2.

In [4]:
pth0 = '/home/maria/IceCube/crosssections/Genie_Maria/npy_genie_with_nu_position_flat_Wcut0/NuMu/NuMu_genie_I3GENIEResultDict'
# pth17 = '/home/maria/IceCube/crosssections/Genie_Maria/Nov5/test_npy_Wcut1.7/NuMu_genie_I3GENIEResultDict'
pthNG = '/home/maria/IceCube/crosssections/NuGen_Maria/'

# read files

In [5]:
def read_my_npy_all(pth, name, pref):
    a = np.load(pth+pref[0]+name)[0]
    
    for i in range(1,len(pref)):
        try:
            b = np.load(pth+pref[i]+name)[0]
            for k in a.keys():
                a[k] = np.concatenate((a[k],b[k]))
        except:
            print 'file', pref[i]+name, 'doesn`t exists '
            
    return a

In [6]:
pref = ['{:0004d}'.format(i) for i in range(0,300)]
v_genie = read_my_npy_all(pth0,'.npy',pref)

In [7]:
v_nugen = np.load(pthNG+'NuGen_vars_Dec10_csms_diff_big_flat_sample_500.npy')[0]

In [8]:
for k in v_genie.keys():
    v_genie[k] = np.array(v_genie[k])
for k in v_nugen.keys():
    v_nugen[k] = np.array(v_nugen[k])

In [9]:
one_weight0 = np.array(v_genie['one_weight'])/float(len(v_genie['one_weight']))

ind_nu0 = np.transpose(np.argwhere(v_genie['neu'] == 14))[0]
ind_nubar0 = np.transpose(np.argwhere(v_genie['neu'] == -14))[0]

one_weight0[ind_nu0] = one_weight0[ind_nu0]/0.7
one_weight0[ind_nubar0] = one_weight0[ind_nubar0]/0.3

In [10]:
one_weight_nugen = np.array(v_nugen['one_weight'])/float(len(v_nugen['one_weight']))

ind_nu0 = np.transpose(np.argwhere(v_nugen['pid'] == 14))[0]
ind_nubar0 = np.transpose(np.argwhere(v_nugen['pid'] == -14))[0]

one_weight_nugen[ind_nu0] = one_weight_nugen[ind_nu0]/0.7
one_weight_nugen[ind_nubar0] = one_weight_nugen[ind_nubar0]/0.3

# selection

In [11]:
nu = 14

ind_nugen_numu = np.transpose(np.argwhere(v_nugen['pid']==nu))[0]
ind_nugen_anti_numu = np.transpose(np.argwhere(v_nugen['pid']==(-1)*nu))[0]

ind_nugen_numu_i1 = ind_nugen_numu[np.transpose(np.argwhere(
    v_nugen['int_type'][ind_nugen_numu]==1))[0]]
ind_nugen_numu_i2 = ind_nugen_numu[np.transpose(np.argwhere(
    v_nugen['int_type'][ind_nugen_numu]==2))[0]]
ind_nugen_anti_numu_i1 = ind_nugen_anti_numu[np.transpose(np.argwhere(
    v_nugen['int_type'][ind_nugen_anti_numu]==1))[0]]
ind_nugen_anti_numu_i2 = ind_nugen_anti_numu[np.transpose(np.argwhere(
    v_nugen['int_type'][ind_nugen_anti_numu]==2))[0]]

In [12]:
def sel_by_key(dict_arrs, keys, keys_vals):
    ind0 = np.arange(0,len(dict_arrs[keys[0]]))
    for i in range(len(keys)):
        ind1 = np.transpose(np.argwhere(dict_arrs[keys[i]][ind0]==keys_vals[i]))[0]
        ind0 = ind0[ind1]
#     print 'final len:', len(ind0)
    return ind0
def ind_nucl(A, nu_type, cc=True):
    
    genie_sel_keys_list = ['dis',
                            'cc',
                            'neu',
                            'hitnuc',
                            'A',
                            'hitqrk',
                            'sea',
                            'charm',
]
    
    ind = []
    for hitnuc in [2212,2112]:
        for ch in [True,False]:
            for sea in [True,False]:
                for i in [-6,-5,-4,-3,-2,-1,1,2,3,4,5,6]:
                        genie_sel_keys_vals = [True,
                                              cc,
                                              nu_type,
                                              hitnuc,
                                              A,
                                              i,
                                              sea,
                                              ch,
                                              ]
                        ind.append(sel_by_key(v_genie,genie_sel_keys_list,genie_sel_keys_vals))

    return ind

In [13]:
ind_NuMu_CC_O_p = ind_nucl(A=16, nu_type=14)
ind_NuMu_NC_O_p = ind_nucl(A=16, nu_type=14, cc=False)
ind_NuMu_Bar_CC_O_p = ind_nucl(A=16, nu_type=-14)
ind_NuMu_Bar_NC_O_p = ind_nucl(A=16, nu_type=-14, cc=False)

ind_NuMu_CC_O_p_conc = np.concatenate(ind_NuMu_CC_O_p)
ind_NuMu_NC_O_p_conc = np.concatenate(ind_NuMu_NC_O_p)
ind_NuMu_Bar_CC_O_p_conc = np.concatenate(ind_NuMu_Bar_CC_O_p)
ind_NuMu_Bar_NC_O_p_conc = np.concatenate(ind_NuMu_Bar_NC_O_p)
ind_all_conc_genie = np.concatenate((ind_NuMu_CC_O_p_conc,ind_NuMu_NC_O_p_conc,
                                     ind_NuMu_Bar_CC_O_p_conc,ind_NuMu_Bar_NC_O_p_conc))

In [14]:
ind_del_genie = np.delete(np.arange(len(v_genie['Ev'])), ind_all_conc_genie)
for k in v_genie.keys():
    v_genie[k] = np.delete(v_genie[k], ind_del_genie)

In [15]:
ind_Q2_cut = np.transpose(np.argwhere(v_genie['Q2'] < 1.))[0]
for k in v_genie.keys():
    v_genie[k] = np.delete(v_genie[k], ind_Q2_cut)

In [16]:
ind_del_genie = []
ind_Q2_cut = []

In [17]:
ind_NuMu_CC_O_p = ind_nucl(A=16, nu_type=14)
ind_NuMu_NC_O_p = ind_nucl(A=16, nu_type=14, cc=False)
ind_NuMu_Bar_CC_O_p = ind_nucl(A=16, nu_type=-14)
ind_NuMu_Bar_NC_O_p = ind_nucl(A=16, nu_type=-14, cc=False)

ind_NuMu_CC_O_p_conc = np.concatenate(ind_NuMu_CC_O_p)
ind_NuMu_NC_O_p_conc = np.concatenate(ind_NuMu_NC_O_p)
ind_NuMu_Bar_CC_O_p_conc = np.concatenate(ind_NuMu_Bar_CC_O_p)
ind_NuMu_Bar_NC_O_p_conc = np.concatenate(ind_NuMu_Bar_NC_O_p)
ind_all_conc_genie = np.concatenate((ind_NuMu_CC_O_p_conc,ind_NuMu_NC_O_p_conc,
                                     ind_NuMu_Bar_CC_O_p_conc,ind_NuMu_Bar_NC_O_p_conc))

In [18]:
lgE_genie = np.log10(v_genie['Ev'])
lgY_genie = np.log10(v_genie['ys'])

lgE_nugen = np.log10(v_nugen['prim_en'])
lgY_nugen = np.log10(v_nugen['y'])

Y_genie = v_genie['ys']
Y_nugen = v_nugen['y']

/home/maria/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log10
  """


In [19]:
one_weight0 = np.array(v_genie['one_weight'])/float(len(v_genie['one_weight']))

ind_nu0 = np.transpose(np.argwhere(v_genie['neu'] == 14))[0]
ind_nubar0 = np.transpose(np.argwhere(v_genie['neu'] == -14))[0]

one_weight0[ind_nu0] = one_weight0[ind_nu0]/0.7
one_weight0[ind_nubar0] = one_weight0[ind_nubar0]/0.3

In [20]:
one_weight_nugen = np.array(v_nugen['one_weight'])/float(len(v_nugen['one_weight']))

ind_nu0 = np.transpose(np.argwhere(v_nugen['pid'] == 14))[0]
ind_nubar0 = np.transpose(np.argwhere(v_nugen['pid'] == -14))[0]

one_weight_nugen[ind_nu0] = one_weight_nugen[ind_nu0]/0.7
one_weight_nugen[ind_nubar0] = one_weight_nugen[ind_nubar0]/0.3

In [21]:
nu = 14

ind_nugen_numu = np.transpose(np.argwhere(v_nugen['pid']==nu))[0]
ind_nugen_anti_numu = np.transpose(np.argwhere(v_nugen['pid']==(-1)*nu))[0]

ind_nugen_numu_i1 = ind_nugen_numu[np.transpose(np.argwhere(
    v_nugen['int_type'][ind_nugen_numu]==1))[0]]
ind_nugen_numu_i2 = ind_nugen_numu[np.transpose(np.argwhere(
    v_nugen['int_type'][ind_nugen_numu]==2))[0]]
ind_nugen_anti_numu_i1 = ind_nugen_anti_numu[np.transpose(np.argwhere(
    v_nugen['int_type'][ind_nugen_anti_numu]==1))[0]]
ind_nugen_anti_numu_i2 = ind_nugen_anti_numu[np.transpose(np.argwhere(
    v_nugen['int_type'][ind_nugen_anti_numu]==2))[0]]

In [22]:
v_genie = []
v_nugen = []

# define type of cross section we are looking at in this notebook

In [23]:
iNuCC_genie = ind_NuMu_CC_O_p_conc
iNuBarCC_genie = ind_NuMu_Bar_CC_O_p_conc

iNuCC_nugen = ind_nugen_numu_i1
iNuBarCC_nugen = ind_nugen_anti_numu_i1

iNuNC_genie = ind_NuMu_NC_O_p_conc
iNuBarNC_genie = ind_NuMu_Bar_NC_O_p_conc

iNuNC_nugen = ind_nugen_numu_i2
iNuBarNC_nugen = ind_nugen_anti_numu_i2

In [24]:
tNuCC = 'NuMu CC'
tNuBarCC = 'NuMu Bar CC'
tNuCC_save = 'NuMu_CC_'
tNuBarCC_save = 'NuMu_Bar_CC_'

tNuNC = 'NuMu NC'
tNuBarNC = 'NuMu Bar NC'
tNuNC_save = 'NuMu_NC_'
tNuBarNC_save = 'NuMu_Bar_NC_'

# save variables and indeces

In [25]:
len(lgE_genie)

21799503

In [26]:
len(lgE_nugen)

50000000

In [27]:
np.save('../../../files/diff_xsec/diff_xsec_vals.npy', [{'lgE_genie':lgE_genie, 'Y_genie':Y_genie, 
                                  'lgE_nugen':lgE_nugen, 'Y_nugen':Y_nugen, 
                                  'one_weight_genie':one_weight0, 'one_weight_nugen':one_weight_nugen,
                                  'iNuCC_genie':iNuCC_genie, 'iNuCC_nugen':iNuCC_nugen,
                                  'iNuBarCC_genie':iNuBarCC_genie, 'iNuBarCC_nugen':iNuBarCC_nugen,
                                  'iNuNC_genie':iNuNC_genie, 'iNuNC_nugen':iNuNC_nugen,
                                  'iNuBarNC_genie':iNuBarNC_genie, 'iNuBarNC_nugen':iNuBarNC_nugen,}])